#Clean and calculate Agreement Rate

In [2]:
LANGUAGE = 'arabic'
import firebase_admin
from firebase_admin import firestore
from datetime import timedelta, datetime
import collections

In [3]:
# Initialize Firestore DB
cred = firebase_admin.credentials.Certificate('../key.json')
default_app = firebase_admin.initialize_app(cred)
# 
cred = firebase_admin.credentials.Certificate('../key.json')
db = firestore.client()
summaries_ref = db.collection(LANGUAGE)
info_ref = db.collection('info')


summaries_ref_data = list(map(lambda t: t.to_dict(), list(summaries_ref.get())))
# Create a collection for the language
info_ref_data = list(map(lambda t: t.to_dict(), list(info_ref.get())))

logs = db.collection('logs')
global_variables_db = db.collection('global_variables')

### Remove answers with short time (1)

In [368]:
summaries_with_enough_time = []
for summary in summaries_ref_data:
    start_time = datetime.fromisoformat(summary['date_start'])
    end_time = datetime.fromisoformat(summary['date_finish'])
    if end_time - start_time > timedelta(seconds=10):
        summaries_with_enough_time.append(summary)

In [335]:
# Remove samples with wrong GPT answer

worker_to_summaries = collections.defaultdict(list)
for summary in summaries_with_enough_time:
    worker_to_summaries[summary['work_id']].append(summary)

results_from_serious_workers = []
# 
for work_id, results in worker_to_summaries.items():
    if len(results) > 10:
        results_from_serious_workers.extend(results)

In [369]:
results_from_serious_workers = summaries_with_enough_time

In [370]:
len(summaries_with_enough_time)

198

In [371]:
# Remove answers with low grade GPT

results_with_answer = [_ for _ in results_from_serious_workers if 'answer' in _ and _['answer'] != ""]
filtered_list = [d for d in results_from_serious_workers if d not in results_with_answer]
filtered_gpt_grades = [_ for _ in results_with_answer if _['gpt_grade'] >= 3]
filtered_gpt_grades.extend(filtered_list)

print(f"Total {len(filtered_gpt_grades)} samples")

Total 198 samples


In [135]:
import pandas as pd

# Assuming your data is in a list called 'data'
data = list_of_dicts

# Find unique n_sample values
unique_n_samples = list(set(item['n_sample'] for item in data))

# Create DataFrame with all columns
df = pd.DataFrame(columns=['fluency_gemini', 'fluency_gpt', 'coherence_gemini', 'coherence_gpt',
                           'relevance_gemini', 'relevance_gpt', 'consistence_gemini', 'consistence_gpt'])

for n_sample in unique_n_samples:
    filtered_data = [item for item in data if item['n_sample'] == n_sample]
    print(len(filtered_data))
    
    # Sort and take the first 3 items for each column
    row_data = {
        'fluency_gemini': sorted([f"{int(item['fluency_gemini'])} ({item['key']})" for item in filtered_data])[:3],
        'fluency_gpt': sorted([f"{int(item['fluency_gpt'])} ({item['key']})" for item in filtered_data])[:3],
        'coherence_gemini': sorted([f"{int(item['coherence_gemini'])} ({item['key']})" for item in filtered_data])[:3],
        'coherence_gpt': sorted([f"{int(item['coherence_gpt'])} ({item['key']})" for item in filtered_data])[:3],
        'relevance_gemini': sorted([f"{int(item['relevance_gemini'])} ({item['key']})" for item in filtered_data])[:3],
        'relevance_gpt': sorted([f"{int(item['relevance_gpt'])} ({item['key']})" for item in filtered_data])[:3],
        'consistence_gemini': sorted([f"{int(item['consistence_gemini'])} ({item['key']})" for item in filtered_data])[:3],
        'consistence_gpt': sorted([f"{int(item['consistence_gpt'])} ({item['key']})" for item in filtered_data])[:3],
        # 'n_sample': n_sample
    }
    
    df = df.append(row_data, ignore_index=True)

print(df)


3


ValueError: invalid literal for int() with base 10: 'x'

In [373]:
df

,fluency_gemini,fluency_gpt,coherence_gemini,coherence_gpt,relevance_gemini,relevance_gpt,consistence_gemini,consistence_gpt
0,[2 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[4 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[3 (A226S9LUL53Q013P888QFVX3V90D22XXM0GWF0Y24Q...,[3 (A226S9LUL53Q013P888QFVX3V90D22XXM0GWF0Y24Q...,[3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[4 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[4 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[4 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...
1,[3 (A234QKV52N964W3W9XHF7WGKW3229FG9O2EK9EY01K...,[3 (A234QKV52N964W3W9XHF7WGKW3229FG9O2EK9EY01K...,[3 (A234QKV52N964W3W9XHF7WGKW3229FG9O2EK9EY01K...,[3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[3 (A234QKV52N964W3W9XHF7WGKW3229FG9O2EK9EY01K...,[3 (A234QKV52N964W3W9XHF7WGKW3229FG9O2EK9EY01K...,[3 (A234QKV52N964W3W9XHF7WGKW3229FG9O2EK9EY01K...,[3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...
2,[3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[2 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[1 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[1 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[1 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[1 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[2 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[1 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...
3,[3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[1 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[2 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[2 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[1 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[1 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[2 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...
4,[2 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[2 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[1 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[2 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[2 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...
...,...,...,...,...,...,...,...,...
62,[3 (A234QKV52N964W3W9XHF7WGKW3229FG9O2EK9EY01K...,[4 (A234QKV52N964W3J94SKDEKIQT3USFNWKH7R8AYOID...,[3 (A234QKV52N964W3UUIU9GZC560KJOC9MKWCVQW1A6T...,[3 (A234QKV52N964W3UUIU9GZC560KJOC9MKWCVQW1A6T...,[3 (A234QKV52N964W3UUIU9GZC560KJOC9MKWCVQW1A6T...,[3 (A234QKV52N964W3J94SKDEKIQT3USFNWKH7R8AYOID...,[2 (A234QKV52N964W3UUIU9GZC560KJOC9MKWCVQW1A6T...,[2 (A234QKV52N964W3UUIU9GZC560KJOC9MKWCVQW1A6T...
63,[3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[4 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[2 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[2 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[2 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[2 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[2 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...,[3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...
64,[4 (A234QKV52N964W35F6NGNVM8KF7N2VXFZTAF4SSG1T...,[4 (A234QKV52N964W35F6NGNVM8KF7N2VXFZTAF4SSG1T...,[3 (A234QKV52N964W3ZCC2DXSD75U3TXQUUCSV9G4BLCY...,[3 (A234QKV52N964W35F6NGNVM8KF7N2VXFZTAF4SSG1T...,[3 (A234QKV52N964W3ZCC2DXSD75U3TXQUUCSV9G4BLCY...,[3 (A234QKV52N964W35F6NGNVM8KF7N2VXFZTAF4SSG1T...,[3 (A234QKV52N964W3ZCC2DXSD75U3TXQUUCSV9G4BLCY...,[3 (A234QKV52N964W35F6NGNVM8KF7N2VXFZTAF4SSG1T...
65,[5 (A234QKV52N964W3BJKPTD2QCDPP6MA6TXZFWVSTWFT...,[5 (A234QKV52N964W3BJKPTD2QCDPP6MA6TXZFWVSTWFT...,[4 (A234QKV52N964W3BJKPTD2QCDPP6MA6TXZFWVSTWFT...,[4 (A234QKV52N964W3BJKPTD2QCDPP6MA6TXZFWVSTWFT...,[3 (A234QKV52N964W3BJKPTD2QCDPP6MA6TXZFWVSTWFT...,[3 (A234QKV52N964W3BJKPTD2QCDPP6MA6TXZFWVSTWFT...,[3 (A234QKV52N964W3BJKPTD2QCDPP6MA6TXZFWVSTWFT...,[3 (A234QKV52N964W3BJKPTD2QCDPP6MA6TXZFWVSTWFT...


In [374]:
df_single_column = df.melt(value_name='stacked').sort_index()

# Drop the variable column (which was holding the original column names)
df_single_column = df_single_column[['stacked']].reset_index(drop=True)

def pad_to_three(lst):
    return lst + [None] * (3 - len(lst))

# Apply the function to each column
for column in df_single_column.columns:
    df_single_column[column] = df_single_column[column].apply(pad_to_three)

print(df_single_column)



                                               stacked
0    [2 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...
1    [3 (A234QKV52N964W3W9XHF7WGKW3229FG9O2EK9EY01K...
2    [3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...
3    [3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...
4    [2 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...
..                                                 ...
531  [2 (A234QKV52N964W3UUIU9GZC560KJOC9MKWCVQW1A6T...
532  [3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...
533  [3 (A234QKV52N964W35F6NGNVM8KF7N2VXFZTAF4SSG1T...
534  [3 (A234QKV52N964W3BJKPTD2QCDPP6MA6TXZFWVSTWFT...
535  [2 (A234QKV52N964W3UUIU9GZC560KJOC9MKWCVQW1A6T...

[536 rows x 1 columns]


In [375]:
df_single_column[:10]

,stacked
0,[2 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...
1,[3 (A234QKV52N964W3W9XHF7WGKW3229FG9O2EK9EY01K...
2,[3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...
3,[3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...
4,[2 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...
5,[3 (A226S9LUL53Q013P888QFVX3V90D22XXM0GWF0Y24Q...
6,[3 (A234QKV52N964W3QGHA0EA0J1U1G9F51GK21XNF0ZB...
7,[4 (A234QKV52N964W3BJKPTD2QCDPP6MA6TXZFWVSTWFT...
8,[4 (A234QKV52N964W3BJKPTD2QCDPP6MA6TXZFWVSTWFT...
9,[3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...


In [376]:
# Function to check if a list has at least two identical numbers
keys = []
def has_identical_numbers(lst):
    try:
        sliced_lst = lst.iloc[0]
    except Exception:
        sliced_lst = lst

    new_lst = [int(_.split(' ')[0]) for _ in sliced_lst if _ is not None]

    # Check if there are exactly two identical numbers
    number_counts = {n: new_lst.count(n) for n in new_lst}
    identical_numbers = [n for n, count in number_counts.items() if count == 2]

    # Find the third number that is at most 1 distance away from the identical numbers
    if len(identical_numbers) == 1:
        identical_num = identical_numbers[0]
        third_number = [n for n in new_lst if n != identical_num]
        if len(third_number) == 1 and abs(third_number[0] - identical_num) <= 1:
            return True
        
    keys.extend([_.split(' ')[1] for _ in sliced_lst if _ is not None])
    return False


df_filtered = pd.DataFrame(df_single_column[df_single_column['stacked'].apply(lambda lst: has_identical_numbers(lst))])

# Display the filtered DataFrame
df_filtered


,stacked
2,[3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...
6,[3 (A234QKV52N964W3QGHA0EA0J1U1G9F51GK21XNF0ZB...
8,[4 (A234QKV52N964W3BJKPTD2QCDPP6MA6TXZFWVSTWFT...
9,[3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...
10,[3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...
...,...
519,[1 (A234QKV52N964W3OQQD2WO8I77XQLX1RXZJRQ3C1J3...
524,[2 (A234QKV52N964W3BJKPTD2QCDPP6MA6TXZFWVSTWFT...
531,[2 (A234QKV52N964W3UUIU9GZC560KJOC9MKWCVQW1A6T...
532,[3 (A226S9LUL53Q013IZPORCT1FAZKXVF6ZGA1DS4I2SH...


In [377]:
len(df_filtered)

197

In [362]:
for _ in range(403):
   print(len(df_filtered.iloc[_][0]))

3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3
3


IndexError: single positional indexer is out-of-bounds

In [378]:
# Function to check if a list has at least two identical numbers
def remove_keys(lst):
    try:
        sliced_lst = lst.iloc[0]
    except Exception:
        sliced_lst = lst
    new_lst = [_.split(' ')[0] for _ in sliced_lst if _ is not None]
    return new_lst


df_filtered = pd.DataFrame(df_filtered['stacked'].apply(lambda lst: remove_keys(lst)))

def pad_to_three(lst):
    return lst + [None] * (3 - len(lst))

# Apply the function to each column
df_filtered['stacked'] = df_filtered['stacked'].apply(pad_to_three)


# Display the filtered DataFrame
df_filtered


,stacked
2,"[3, 3, 4]"
6,"[3, 3, 4]"
8,"[4, 4, 5]"
9,"[3, 4, 4]"
10,"[3, 4, 4]"
...,...
519,"[1, 2, 2]"
524,"[2, 2, 3]"
531,"[2, 3, 3]"
532,"[3, 3, 4]"


In [379]:
len(keys)

993

In [380]:
# Calculate agreement rate

import numpy as np
df =df_filtered

# Convert the column of lists into a 3xN matrix
matrix = np.array(df['stacked'].tolist()).T

# Function to replace None with *
def replace_none_with_star(x):
    return " ".join(str(i) if i is not None else "*" for i in x)

# Convert each row of the matrix to the desired format
formatted_data = tuple(replace_none_with_star(row) for row in matrix)


try:
    import numpy as np
except ImportError:
    np = None


def nominal_metric(a, b):
    return a != b


def interval_metric(a, b):
    return (a-b)**2


def ratio_metric(a, b):
    return ((a-b)/(a+b))**2


def krippendorff_alpha(data, metric=interval_metric, force_vecmath=False, convert_items=float, missing_items=None):
    '''
    Calculate Krippendorff's alpha (inter-rater reliability):
    
    data is in the format
    [
        {unit1:value, unit2:value, ...},  # coder 1
        {unit1:value, unit3:value, ...},   # coder 2
        ...                            # more coders
    ]
    or 
    it is a sequence of (masked) sequences (list, numpy.array, numpy.ma.array, e.g.) with rows corresponding to coders and columns to items
    
    metric: function calculating the pairwise distance
    force_vecmath: force vector math for custom metrics (numpy required)
    convert_items: function for the type conversion of items (default: float)
    missing_items: indicator for missing items (default: None)
    '''
    
    # number of coders
    m = len(data)
    
    # set of constants identifying missing values
    if missing_items is None:
        maskitems = []
    else:
        maskitems = list(missing_items)
    if np is not None:
        maskitems.append(np.ma.masked_singleton)
    
    # convert input data to a dict of items
    units = {}
    for d in data:
        try:
            # try if d behaves as a dict
            diter = d.items()
        except AttributeError:
            # sequence assumed for d
            diter = enumerate(d)
            
        for it, g in diter:
            if g not in maskitems:
                try:
                    its = units[it]
                except KeyError:
                    its = []
                    units[it] = its
                its.append(convert_items(g))


    units = dict((it, d) for it, d in units.items() if len(d) > 1)  # units with pairable values
    n = sum(len(pv) for pv in units.values())  # number of pairable values
    
    if n == 0:
        raise ValueError("No items to compare.")
    
    np_metric = (np is not None) and ((metric in (interval_metric, nominal_metric, ratio_metric)) or force_vecmath)
    
    Do = 0.
    for grades in units.values():
        if np_metric:
            gr = np.asarray(grades)
            Du = sum(np.sum(metric(gr, gri)) for gri in gr)
        else:
            Du = sum(metric(gi, gj) for gi in grades for gj in grades)
        Do += Du/float(len(grades)-1)
    Do /= float(n)

    if Do == 0:
        return 1.

    De = 0.
    for g1 in units.values():
        if np_metric:
            d1 = np.asarray(g1)
            for g2 in units.values():
                De += sum(np.sum(metric(d1, gj)) for gj in g2)
        else:
            for g2 in units.values():
                De += sum(metric(gi, gj) for gi in g1 for gj in g2)
    De /= float(n*(n-1))

    return 1.-Do/De if (Do and De) else 1.



missing = '*' # indicator for missing values
array = [d.split() for d in formatted_data]  # convert to 2D list of string items

print("nominal metric: %.3f" % krippendorff_alpha(array, nominal_metric, missing_items=missing))
print("interval metric: %.3f" % krippendorff_alpha(array, interval_metric, missing_items=missing))

nominal metric: -0.050
interval metric: 0.428


In [366]:
# Get filtered documents

clean_keys = [key[1:-1] for key in keys if keys.count(key) > 1]

In [302]:
clean_documents = [doc for doc  in summaries_ref_data if doc['key'] in clean_keys]

In [305]:
len(summaries_ref_data)

219

In [304]:
# Assuming your data is a list of JSONs called 'data'
data = summaries_ref_data

# Step 1: Use a set to track unique combinations of (work_n, sample_n)
unique_work_sample_pairs = set()
filtered_data = []

# Step 2: Iterate over the data and keep only unique (work_n, sample_n) combinations
for item in data:
    work_sample_pair = (item['work_id'], item['n_sample'])
    
    if work_sample_pair not in unique_work_sample_pairs:
        unique_work_sample_pairs.add(work_sample_pair)
        filtered_data.append(item)

# Result: 'filtered_data' will only contain one document for each (work_n, sample_n) pair
print(filtered_data)


[{'gpt_prediction': 'הסיכום הוא על הבמאי ניר טויב והמאבק שעבר בכדי להפיק ולשדר את סרטו "ממלכת הסוד". הסרט נפסל להקרנה על ידי הצנזורה הצבאית ונדרש מאבק עיקש כדי לשחררו. טויב התעקש על הזכות להמחיש את האמת בסרטו באמצעות אודיו-ויזואליים. הוא ניהל גם את מחלתו באופן חדשני ולוחמני והיה דוגמה חיה לאופן שבו יכולים לשפר תפקוד ולהאריך ימים. הסיכום מציין כי הצנזורה פעלה משיקולים זרים בפסילת הסרט.', 'answer': 'לא ידוע', 'relevance_gpt': '2', 'gpt_question': 'האם הטלוויזיה של השותפות סבן־ורטהיים תהיה שונה מהטלוויזיה של פישמן־שטראוס?', 'summary': 'היוצר התיעודי ניר טויב, שנפטר השבוע, האיר בסרטיו באופן חושפני וביקורתי כמה מן הפרשות הצבאיות המעצבות את חיינו', 'gpt_grade': 4, 'consistence_gpt': '2', 'work_id': '1A', 'assignmentId': '10', 'task': 1, 'key': '1A101', 'hit_id': '10', 'date_start': '2024-09-14 10:57:27.173781', 'coherence_gpt': '4', 'n_sample': 16, 'date_finish': '2024-09-14 10:58:40.370779', 'original_gpt_grade': '4', 'fluency_gemini': '3', 'relevance_gemini': '2', 'fluency_gpt': '2', 'cohe

In [262]:
len(filtered_data)

198

In [8]:
# Initialize Firestore DB
LANGUAGE = ''

cred = firebase_admin.credentials.Certificate('../key.json')
# default_app = firebase_admin.initialize_app(cred)
# 
cred = firebase_admin.credentials.Certificate('../key.json')
db = firestore.client()
summaries_ref = db.collection(LANGUAGE)
info_ref = db.collection('info')


summaries_ref_data2 = list(map(lambda t: t.to_dict(), list(summaries_ref.get())))
# Create a collection for the language
info_ref_data = list(map(lambda t: t.to_dict(), list(info_ref.get())))

logs = db.collection('logs')
global_variables_db = db.collection('global_variables')

In [9]:
summaries_ref_data += summaries_ref_data2

In [10]:
len(summaries_ref_data)

769

In [11]:
language_collection = db.collection('arabic')
# summary_ref = info_ref.document(f"{LANGUAGE}")
# summary_ref.set(n_sample_counts)



In [12]:
summaries_ref_data[0]

{'gpt_prediction': 'أكد المتحدث باسم البيت الأبيض الالتزام بحماية أعضاء الناتو وأشار إلى أن الولايات المتحدة ستلتزم بالحماية فقط للدول التي تفي بالتزاماتها تجاه الولايات المتحدة. ورغم عدم تعليق البيت الأبيض على تصريحات ترامب، إلا أن المتحدث أكد أن التزام الولايات المتحدة نحو دول الناتو قوي وأن الرئيس جدد الالتزام قبل أسبوعين. وتمارس الولايات المتحدة ضغوطًا على الدول الأعضاء في الناتو لزيادة نفقاتها العسكرية. وفي ضوء تفاقم التوتر مع روسيا، أشار ترامب إلى إمكانية توقف الولايات المتحدة عن تقديم الحماية للدول الأعضاء في حالة العدوان الروسي على دول في حوض البلطيق. وأشار ترامب إلى أن الولايات المتحدة ستعيد تقييم اتفاقياتها الدفاعية لتلزم الحلفاء بدفع تكاليف أي عمل عسكري.',
 'answer': 'الثلاثاء',
 'relevance_gpt': '2',
 'gpt_question': 'متى توفي النجم البريطاني السير روجر مور؟',
 'summary': 'قال دونالد ترامب، مرشح الرئاسة الأمريكية عن الحزب الجمهوري، إنه في حال انتخابه قد يلغي "ضمان الحماية الأمريكية" لدول حلف شمال الأطلسي "الناتو".',
 'gpt_grade': 4,
 'consistence_gpt': '4',
 'work_id': 'A1K

In [13]:
for doc in summaries_ref_data:
    doc_ref = language_collection.document(doc['key'])
    doc_ref.set(doc)

In [320]:
# Update DB 

from collections import defaultdict

# Example list of dictionaries (your data)
data = summaries_with_enough_time

# Step 1: Create a dictionary to store the count of each n_sample
n_sample_counts = defaultdict(int)

# Step 2: Count the occurrences of each n_sample
for entry in data:
    n_sample = str(entry['n_sample'])
    n_sample_counts[n_sample] += 1

# Convert to a regular dictionary if you don't want the defaultdict behavior
n_sample_counts = dict(n_sample_counts)

# Show the resulting dictionary
print(n_sample_counts)

{'16': 2, '25': 2, '18': 2, '3': 1, '26': 2, '65': 2, '28': 3, '70': 2, '88': 4, '1': 2, '7': 4, '44': 2, '12': 3, '85': 2, '61': 3, '79': 2, '30': 2, '77': 3, '86': 2, '47': 2, '55': 2, '94': 2, '36': 2, '4': 3, '2': 2, '17': 3, '96': 3, '84': 2, '87': 1, '64': 3, '73': 2, '67': 1, '29': 3, '33': 3, '58': 3, '31': 4, '75': 2, '11': 2, '52': 3, '46': 1, '90': 2, '98': 2, '59': 3, '32': 1, '8': 2, '43': 2, '97': 2, '40': 4, '76': 4, '48': 3, '72': 3, '42': 2, '21': 3, '34': 2, '20': 2, '13': 4, '81': 2, '6': 3, '15': 2, '56': 4, '10': 3, '60': 3, '92': 4, '41': 4, '35': 3, '62': 1, '49': 2, '27': 3, '50': 3, '95': 2, '89': 3, '51': 1, '68': 3, '63': 2, '54': 2, '38': 3, '66': 2, '22': 1, '93': 2, '71': 2, '23': 4, '5': 2, '74': 1, '14': 1, '24': 1, '37': 2, '57': 1}


In [214]:
clean_documents[0]['n_sample']

47

In [215]:
pd.DataFrame(clean_documents).to_csv(f"/Users/itaimondshine/PycharmProjects/NLP/eval_metrics/evaluations/results/human/documents/{LANGUAGE}.csv")

In [321]:
additional_values = {str(i):0 for i in range(0,100) if str(i) not in n_sample_counts }

In [322]:
n_sample_counts.update(additional_values)

In [323]:
n_sample_counts

{'16': 2,
 '25': 2,
 '18': 2,
 '3': 1,
 '26': 2,
 '65': 2,
 '28': 3,
 '70': 2,
 '88': 4,
 '1': 2,
 '7': 4,
 '44': 2,
 '12': 3,
 '85': 2,
 '61': 3,
 '79': 2,
 '30': 2,
 '77': 3,
 '86': 2,
 '47': 2,
 '55': 2,
 '94': 2,
 '36': 2,
 '4': 3,
 '2': 2,
 '17': 3,
 '96': 3,
 '84': 2,
 '87': 1,
 '64': 3,
 '73': 2,
 '67': 1,
 '29': 3,
 '33': 3,
 '58': 3,
 '31': 4,
 '75': 2,
 '11': 2,
 '52': 3,
 '46': 1,
 '90': 2,
 '98': 2,
 '59': 3,
 '32': 1,
 '8': 2,
 '43': 2,
 '97': 2,
 '40': 4,
 '76': 4,
 '48': 3,
 '72': 3,
 '42': 2,
 '21': 3,
 '34': 2,
 '20': 2,
 '13': 4,
 '81': 2,
 '6': 3,
 '15': 2,
 '56': 4,
 '10': 3,
 '60': 3,
 '92': 4,
 '41': 4,
 '35': 3,
 '62': 1,
 '49': 2,
 '27': 3,
 '50': 3,
 '95': 2,
 '89': 3,
 '51': 1,
 '68': 3,
 '63': 2,
 '54': 2,
 '38': 3,
 '66': 2,
 '22': 1,
 '93': 2,
 '71': 2,
 '23': 4,
 '5': 2,
 '74': 1,
 '14': 1,
 '24': 1,
 '37': 2,
 '57': 1,
 '0': 0,
 '9': 0,
 '19': 0,
 '39': 0,
 '45': 0,
 '53': 0,
 '69': 0,
 '78': 0,
 '80': 0,
 '82': 0,
 '83': 0,
 '91': 0,
 '99': 0}

In [324]:
summary_ref = info_ref.document(f"{LANGUAGE}")

In [325]:
summary_ref.set(n_sample_counts)

update_time {
  seconds: 1726482702
  nanos: 73662000
}

In [218]:
new_summaries_ref = db.collection(f"{LANGUAGE}_new")


In [219]:
for doc in clean_documents:
    new_summaries_ref.add(doc, document_id=doc['key'])

In [383]:
a1 = [_ for _ in summaries_with_enough_time if _['work_id'] == '1D']

In [384]:
len(a1)

0

In [249]:
summaries_with_enough_time = []
for summary in a1:
    start_time = datetime.fromisoformat(summary['date_start'])
    end_time = datetime.fromisoformat(summary['date_finish'])
    if end_time - start_time > timedelta(seconds=30):
        summaries_with_enough_time.append(summary)

In [250]:
len(summaries_with_enough_time)

63

# Save documents

In [4]:
# Initialize Firestore DB
LANGUAGE = 'yoruba'
cred = firebase_admin.credentials.Certificate('../key.json')
# default_app = firebase_admin.initialize_app(cred)
# 
cred = firebase_admin.credentials.Certificate('../key.json')
db = firestore.client()
summaries_ref = db.collection(LANGUAGE)
info_ref = db.collection('info')


summaries_ref_data = list(map(lambda t: t.to_dict(), list(summaries_ref.get())))
# Create a collection for the language
info_ref_data = list(map(lambda t: t.to_dict(), list(info_ref.get())))

logs = db.collection('logs')
global_variables_db = db.collection('global_variables')


In [6]:
len(summaries_ref_data)

245

In [148]:
spanish2 = summaries_ref_data

In [142]:
len(spanish2)

236

In [150]:
spanish1 = summaries_ref_data

In [151]:
import pandas as pd
spanish1_df = pd.DataFrame(spanish1)
spanish2_df = pd.DataFrame(spanish2)

In [152]:
spanish = pd.concat([spanish2_df, spanish1_df], axis=0)

In [153]:
spanish = spanish.sort_values(by='n_sample')

In [157]:
spanish2_df

,gpt_prediction,answer,relevance_gpt,gpt_question,summary,gpt_grade,consistence_gpt,work_id,assignmentId,task,...,n_sample,date_finish,original_gpt_grade,fluency_gemini,relevance_gemini,fluency_gpt,coherence_gemini,location,gemini_prediction,consistence_gemini
0,"Ọrọ ti a kọ fun ijade yẹ ni pe Koda Alhasan, t...",beeni,4,Unai Emery ti sọ oju abẹ niko nigba to sọ pe k...,"Sẹnetọ Aisha Alhasan, ti gbogbo ènìyàn mọ si M...",4,4,A165LB5CPPQE6M,3NC5L260MONFV8AN9JG8SG7IYWTFO4,0,...,238,2024-09-21 21:44:31.700797,4,2,3,4,2,"Lagos, Nigeria.",**Taraba: Akọwe Ọ̀rọ̀ Obìnrin Ti Gba Fọọmu Ti ...,3
1,Ọjọ Kọkanlelogun Osu Kọkanla ni Davido maa n s...,"Beeni, Aisha Alhasan ti gbogbo ènìyàn mọ si Ma...",3,Aisha Alhasan ti gbogbo ènìyàn mọ si Mama Tara...,Bi o tilẹ́ jẹ pe ayẹyẹ ọjọ ibi ilu mọọka olori...,4,3,A165LB5CPPQE6M,3NC5L260MONFV8AN9JG8SG7IYWTFO4,1,...,256,2024-09-21 21:46:50.150079,4,3,5,4,2,"Lagos, Nigeria.","Ifẹ Ọlọrun yóò máa bà á jẹ fun ọ, mọ́lẹ̀bì,\nN...",4
2,Ọrọ ti a fun ni pe Cristiano Ronaldo ti padabo...,Beeni,3,Bukọla Saraki ti ni otitọ kankan ninu ẹsun ti ...,Ipadabọ agbabọọlu si ẹgbẹ to ti figba kan gba ...,4,3,A165LB5CPPQE6M,3NC5L260MONFV8AN9JG8SG7IYWTFO4,10,...,337,2024-09-21 22:02:00.333930,4,5,4,4,5,"Lagos, Nigeria.",Ipade Ronaldo pẹlu Manchester United. Bawo ni ...,5
3,Translation:\n\nThere is a controversy surroun...,Beeni,4,Davido ranpẹ ko wọle de ni ilu mọọka olorin ta...,"Otun ti ilu Ibadan, Lekan Balogun ti ni oye ni...",4,3,A165LB5CPPQE6M,3NC5L260MONFV8AN9JG8SG7IYWTFO4,2,...,326,2024-09-21 21:48:24.780502,4,1,1,4,1,"Lagos, Nigeria.",Balogun sọ pé ọ̀gọ́jì ọdún yìí ní tí òun ti j...,1
4,Ọrọ ti a fun ni pe awọn oludije akọkọ lati ilẹ...,Beeni,5,Lekan Balogun ti ni oye ni ilu Ibadan kii ṣe f...,Aarẹ Muhammadu Buhari ti ransẹ ayọ sawọn oludi...,4,5,A165LB5CPPQE6M,3NC5L260MONFV8AN9JG8SG7IYWTFO4,3,...,276,2024-09-21 21:49:15.506046,4,5,5,5,5,"Lagos, Nigeria.","Aṣaaju, Naijiria kọ́kọ́ kọ́pa nínú ìdíje ìdàra...",5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,"Ọrọ ti a fun kọ akopọ ni pe Bukola Saraki, góm...",8 Members of the Redeemed Christian Church of ...,4,Kini ohun ti jọ iroyin to lu ori ayelujara pe ...,BBC Yoruba se abẹwo si ilẹ Sẹnatọ Bukọla Sarak...,4,4,A7C7BKFSC8417,3Y9N9SS8LYCTBZ35ZF1QGD6MQDWD3G,5,...,334,2024-09-20 17:06:14.338131,4,4,4,4,4,"LAGOS,NIGERIA",**Ilé Olusola Saraki**\n\nNígbà tí ìgbà alẹ̀ ...,4
232,Ọrọ ti a fun kọ akopọ jẹ pe Mohammed Umar jẹ ọ...,He lives in Ikoyi in Lagos state,4,Saraki ti wa ni agbegbe Ikoyi ni Eko?,Mohammed Umar ni ijọba Naijiria yan gẹgẹ bi ol...,4,4,A7C7BKFSC8417,3Y9N9SS8LYCTBZ35ZF1QGD6MQDWD3G,6,...,316,2024-09-20 17:09:27.630245,4,4,4,4,4,"LAGOS,NIGERIA",Ijoba ti sọ Mohammed Umar di olori EFCC tuntun...,4
233,Ọpọlọpọ awọn ọṣẹ ni ojo naa ti o ṣe lasiko ti ...,Mohammed Umar is the newly appointed leader of...,3,Mohammed Umar ni ijọba Naijiria yan gẹgẹ bi ol...,Ojo to rọ lọjọ aje ni ilu Eko ti da ṣugbọn ọwọ...,4,3,A7C7BKFSC8417,3Y9N9SS8LYCTBZ35ZF1QGD6MQDWD3G,7,...,339,2024-09-20 17:11:45.512666,4,4,4,4,4,"LAGOS,NIGERIA",**Akopọ**\n\n* Ọpọlọpọ ọṣẹ ni ọjọ to baje kọlù...,4
234,"Ọba Oloyede, ni ipinlẹ naa, ti sọ pe o ti mọ t...",The rain that fell in Lagos on Monday,4,Kini ipa ti o fi silẹ ṣi n ja ranyinranyin ni ...,Awọn ọmọ Naijiria lori ayelujara ti fi ero wọn...,4,4,A7C7BKFSC8417,3Y9N9SS8LYCTBZ35ZF1QGD6MQDWD3G,8,...,229,2024-09-20 17:15:33.515638,4,2,2,4,2,"LAGOS,NIGERIA",Ijoba ti da awọn olori ẹgbẹ́ NURTW lẹ̀yìn pe k...,2


In [154]:
# import pandas as pd
# df = pd.DataFrame(summaries_ref_data)

In [158]:
df = df.sort_values(by='n_sample')

KeyError: 'n_sample'

In [424]:
# df.to_excel("/Users/itaimondshine/PycharmProjects/NLP/eval_metrics/evaluations/results/human/documents/hebrew.xlsx", index=False)

In [425]:
# Check Agreement Rate
# Check Correlation to Gemini 
# Check Correlation to Other metrics 

In [426]:
# df = pd.read_excel("/Users/itaimondshine/PycharmProjects/NLP/eval_metrics/evaluations/results/human/documents/hebrew.xlsx")

In [427]:
# n_sample_counts = df.groupby('n_sample').size()
# 
# # Identify and print `n_sample` values with more than 3 rows
# n_samples_with_more_than_3_rows = n_sample_counts[n_sample_counts > 3]
# 
# print("n_sample values with more than 3 rows:")
# panprint(n_samples_with_more_than_3_rows)

In [65]:
criteria_columns = [
    'coherence_gpt', 'consistence_gpt', 'relevance_gpt', 'fluency_gpt',
    'coherence_gemini', 'consistence_gemini', 'relevance_gemini', 'fluency_gemini', 'n_sample'
]

In [66]:
spanish = spanish[criteria_columns]

In [705]:
spanish = spanish.dropna()

In [161]:
len(spanish)

481

In [706]:
set(spanish['coherence_gpt'])

{'1', '2', '3', '4', '5'}

In [182]:
import pandas as pd
import numpy as np



# Define criteria columns
criteria_columns = [
    'coherence_gpt', 'consistence_gpt', 'relevance_gpt', 'fluency_gpt',
    'coherence_gemini', 'consistence_gemini', 'relevance_gemini', 'fluency_gemini'
]

# Function to check if a row has at least 3 criteria that differ by at least 2
def filter_rows(group):
    if len(group) <= 1:
        return group
    
    rows_to_keep = []
    
    for i, row in group.iterrows():
        # Compare current row with others in the group
        other_rows = group.drop(i)
        mismatches = 0
        
        for _, other_row in other_rows.iterrows():
            # Count mismatches for this comparison
            for col in criteria_columns:
                print(col)
                if pd.notna(row[col]) and pd.notna(other_row[col]):
                    if abs(int(row[col]) - int(other_row[col])) >= 2:
                        mismatches += 1
        
        # If at least 3 criteria mismatch for any comparison, discard this row
        if mismatches <3  * len(other_rows):
            rows_to_keep.append(row)
    
    return pd.DataFrame(rows_to_keep)

# Apply the filter function to each group
df_filtered = spanish.groupby('n_sample').apply(filter_rows).reset_index(drop=True)

# Display the filtered DataFrame
print(df_filtered)


coherence_gpt
consistence_gpt
relevance_gpt
fluency_gpt
coherence_gemini
consistence_gemini
relevance_gemini
fluency_gemini
coherence_gpt
consistence_gpt
relevance_gpt
fluency_gpt
coherence_gemini
consistence_gemini
relevance_gemini
fluency_gemini
coherence_gpt
consistence_gpt
relevance_gpt
fluency_gpt
coherence_gemini
consistence_gemini
relevance_gemini
fluency_gemini
coherence_gpt
consistence_gpt
relevance_gpt
fluency_gpt
coherence_gemini
consistence_gemini
relevance_gemini
fluency_gemini
coherence_gpt
consistence_gpt
relevance_gpt
fluency_gpt
coherence_gemini
consistence_gemini
relevance_gemini
fluency_gemini
coherence_gpt
consistence_gpt
relevance_gpt
fluency_gpt
coherence_gemini
consistence_gemini
relevance_gemini
fluency_gemini
coherence_gpt
consistence_gpt
relevance_gpt
fluency_gpt
coherence_gemini
consistence_gemini
relevance_gemini
fluency_gemini
coherence_gpt
consistence_gpt
relevance_gpt
fluency_gpt
coherence_gemini
consistence_gemini
relevance_gemini
fluency_gemini
coherenc

In [183]:
len(set(df_filtered['n_sample']))

135

In [184]:
# df_filtered.to_excel("/Users/itaimondshine/PycharmProjects/NLP/eval_metrics/evaluations/results/human/documents/yoruba_v1.xlsx")

In [185]:
# df_filtered

In [186]:
columns_to_convert = ['coherence_gpt', 'consistence_gpt', 'relevance_gpt', 'fluency_gpt', 
                      'coherence_gemini', 'consistence_gemini', 'relevance_gemini', 'fluency_gemini']

# Convert only the specified columns to integers
df_filtered = df_filtered.fillna(0)

df_filtered[columns_to_convert] = df_filtered[columns_to_convert].astype(int)

In [187]:
df_filtered

,gpt_prediction,answer,relevance_gpt,gpt_question,summary,gpt_grade,consistence_gpt,work_id,assignmentId,task,...,n_sample,date_finish,original_gpt_grade,fluency_gemini,relevance_gemini,fluency_gpt,coherence_gemini,location,gemini_prediction,consistence_gemini
0,ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...,0,4,0,Ijọba apaapọ orilẹede Naijiria ni wọn yoo se i...,0.0,4,A32OWZMIVWENYE,39U1BHVTDLS0SOZ2L515LHO8NHQ3TP,6.0,...,4.0,2024-09-01 15:54:15.977230,0,4,5,4,5,0,**Òrọ Tí A Túmọ̀**\n\n* Igbakeji Aarẹ Naijiria...,4
1,ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...,0,4,0,Ijọba apaapọ orilẹede Naijiria ni wọn yoo se i...,0.0,4,A2ON3JHU4XJ11Y,33F859I566EFQACK04ZMPHJW0F8HBK,10.0,...,4.0,2024-09-01 01:00:53.956401,0,5,4,4,4,0,**Òrọ Tí A Túmọ̀**\n\n* Igbakeji Aarẹ Naijiria...,4
2,ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...,0,5,0,Ijọba apaapọ orilẹede Naijiria ni wọn yoo se i...,0.0,4,A32OWZMIVWENYE,39U1BHVTDLS0SOZ2L515LHO8NHQ3TP,5.0,...,4.0,2024-09-01 15:53:18.326913,0,5,4,4,5,0,**Òrọ Tí A Túmọ̀**\n\n* Igbakeji Aarẹ Naijiria...,4
3,Ọrọ ti a ti kọ fun yin ni nipa iya kan to pa ọ...,0,4,0,Orisirisi iwa isekupani lo n waye ni Naijiria ...,0.0,4,A38NAQ9O99MMIH,3CP1TO84PT29M4STG952ZRD210425M,2.0,...,5.0,2024-09-01 06:57:16.059175,0,4,4,4,4,0,**Awọn Ọrọ Yorùbá:**\n\n* Iroyin (ìròyìn)\n* K...,4
4,Ọrọ ti a ti kọ fun yin ni nipa iya kan to pa ọ...,0,4,0,Orisirisi iwa isekupani lo n waye ni Naijiria ...,0.0,3,AVW7ZRR5PLRFW,3PJUZCGDJ6H396WUZGRKLYV79WF98N,4.0,...,5.0,2024-09-01 13:25:21.954945,0,4,2,2,4,0,**Awọn Ọrọ Yorùbá:**\n\n* Iroyin (ìròyìn)\n* K...,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...,Beeni.,3,Awọn ọrọ ni ki o ṣe iru awọn ohun ti o jẹ ohun...,Wọn bi Aarẹ Muhammadu Buhari ni idile ẹya Fula...,4.0,4,A165LB5CPPQE6M,3W92K5RLWUIIJVQV4JPD1WAD5VGV57,7.0,...,341.0,2024-09-21 22:35:57.353065,I would rate the given answer a 4. It correctl...,3,3,4,2,"Lagos, Nigeria.",**Àgbà**\n\n* Ààrẹ\n* Oloye\n* Ọgagun\n* Ọmọ o...,3
315,Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...,Beeni,3,Maesaiah Thabane ti fi ẹsun ipaniyan kan loril...,Wọn bi Aarẹ Muhammadu Buhari ni idile ẹya Fula...,4.0,3,A165LB5CPPQE6M,3W92K5RLWUIIJVQV4JPD1WAD5VGV57,3.0,...,341.0,2024-09-21 22:30:17.507613,4,4,3,4,4,"Lagos, Nigeria.",**Àgbà**\n\n* Ààrẹ\n* Oloye\n* Ọgagun\n* Ọmọ o...,4
316,Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...,The representative of the pope and Bishop atte...,4,Kini orisun aṣoju Poopu ati aadọta bisọbu ni y...,Wọn bi Aarẹ Muhammadu Buhari ni idile ẹya Fula...,3.0,4,A2A6H80LVCI4U2,3GNA64GUZE5QENUEIKV3V9B5O3Z5QL,1.0,...,341.0,2024-09-21 22:08:04.740232,3,4,4,5,4,"LAGOS,NIGERIA",**Àgbà**\n\n* Ààrẹ\n* Oloye\n* Ọgagun\n* Ọmọ o...,4
317,Ọdun miran tun ti ko fun ilumọọka olorin juju ...,King AbdulRasheed and the Cheif priest are qua...,4,Kini o nipa ija ti wa laarin Oba Abdulrasheed ...,"Yoruba ni ọdọọdun la n ri orogbo, ọdọọdun la n...",3.0,4,A7C7BKFSC8417,3TMSXRD2X61WA2PHT6SXK8GAZ8Y1WT,7.0,...,342.0,2024-09-20 13:44:29.542207,I would rate the given answer a 3. While it me...,1,2,4,2,"LAGOS,NIGERIA","Ìgbà ọ̀tun tún ti wá fún ọ̀gá ọ̀rọ̀ àgbà, olór...",2


In [205]:
df_filtered = df_filtered.groupby('n_sample').filter(lambda x: len(x) >= 2)

# Display the filtered DataFrame
print(df_filtered)

                                        gpt_prediction  \
0    ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...   
1    ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...   
2    ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...   
3    Ọrọ ti a ti kọ fun yin ni nipa iya kan to pa ọ...   
4    Ọrọ ti a ti kọ fun yin ni nipa iya kan to pa ọ...   
..                                                 ...   
298  Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...   
299  Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...   
300  Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...   
301  Ọdun miran tun ti ko fun ilumọọka olorin juju ...   
302  Ọdun miran tun ti ko fun ilumọọka olorin juju ...   

                                                answer  relevance_gpt  \
0                                                    0              4   
1                                                    0              4   
2                                                    0              5   
3          

In [189]:
# # df_filtered = df_filtered.sort_values(by='n_sample')
# df_filtered = pd.read_excel("/Users/itaimondshine/PycharmProjects/NLP/eval_metrics/evaluations/results/human/documents/ukrainian.xlsx")

In [207]:
len(set(df_filtered['n_sample']))

119

In [208]:
df_filtered.to_ex

,gpt_prediction,answer,relevance_gpt,gpt_question,summary,gpt_grade,consistence_gpt,work_id,assignmentId,task,...,n_sample,date_finish,original_gpt_grade,fluency_gemini,relevance_gemini,fluency_gpt,coherence_gemini,location,gemini_prediction,consistence_gemini
0,ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...,0,4,0,Ijọba apaapọ orilẹede Naijiria ni wọn yoo se i...,0.0,4,A32OWZMIVWENYE,39U1BHVTDLS0SOZ2L515LHO8NHQ3TP,6.0,...,4.0,2024-09-01 15:54:15.977230,0,4,5,4,5,0,**Òrọ Tí A Túmọ̀**\n\n* Igbakeji Aarẹ Naijiria...,4
1,ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...,0,4,0,Ijọba apaapọ orilẹede Naijiria ni wọn yoo se i...,0.0,4,A2ON3JHU4XJ11Y,33F859I566EFQACK04ZMPHJW0F8HBK,10.0,...,4.0,2024-09-01 01:00:53.956401,0,5,4,4,4,0,**Òrọ Tí A Túmọ̀**\n\n* Igbakeji Aarẹ Naijiria...,4
2,ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...,0,5,0,Ijọba apaapọ orilẹede Naijiria ni wọn yoo se i...,0.0,4,A32OWZMIVWENYE,39U1BHVTDLS0SOZ2L515LHO8NHQ3TP,5.0,...,4.0,2024-09-01 15:53:18.326913,0,5,4,4,5,0,**Òrọ Tí A Túmọ̀**\n\n* Igbakeji Aarẹ Naijiria...,4
3,Ọrọ ti a ti kọ fun yin ni nipa iya kan to pa ọ...,0,4,0,Orisirisi iwa isekupani lo n waye ni Naijiria ...,0.0,4,A38NAQ9O99MMIH,3CP1TO84PT29M4STG952ZRD210425M,2.0,...,5.0,2024-09-01 06:57:16.059175,0,4,4,4,4,0,**Awọn Ọrọ Yorùbá:**\n\n* Iroyin (ìròyìn)\n* K...,4
4,Ọrọ ti a ti kọ fun yin ni nipa iya kan to pa ọ...,0,4,0,Orisirisi iwa isekupani lo n waye ni Naijiria ...,0.0,3,AVW7ZRR5PLRFW,3PJUZCGDJ6H396WUZGRKLYV79WF98N,4.0,...,5.0,2024-09-01 13:25:21.954945,0,4,2,2,4,0,**Awọn Ọrọ Yorùbá:**\n\n* Iroyin (ìròyìn)\n* K...,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...,Beeni.,3,Awọn ọrọ ni ki o ṣe iru awọn ohun ti o jẹ ohun...,Wọn bi Aarẹ Muhammadu Buhari ni idile ẹya Fula...,4.0,4,A165LB5CPPQE6M,3W92K5RLWUIIJVQV4JPD1WAD5VGV57,7.0,...,341.0,2024-09-21 22:35:57.353065,I would rate the given answer a 4. It correctl...,3,3,4,2,"Lagos, Nigeria.",**Àgbà**\n\n* Ààrẹ\n* Oloye\n* Ọgagun\n* Ọmọ o...,3
299,Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...,Beeni,3,Maesaiah Thabane ti fi ẹsun ipaniyan kan loril...,Wọn bi Aarẹ Muhammadu Buhari ni idile ẹya Fula...,4.0,3,A165LB5CPPQE6M,3W92K5RLWUIIJVQV4JPD1WAD5VGV57,3.0,...,341.0,2024-09-21 22:30:17.507613,4,4,3,4,4,"Lagos, Nigeria.",**Àgbà**\n\n* Ààrẹ\n* Oloye\n* Ọgagun\n* Ọmọ o...,4
300,Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...,The representative of the pope and Bishop atte...,4,Kini orisun aṣoju Poopu ati aadọta bisọbu ni y...,Wọn bi Aarẹ Muhammadu Buhari ni idile ẹya Fula...,3.0,4,A2A6H80LVCI4U2,3GNA64GUZE5QENUEIKV3V9B5O3Z5QL,1.0,...,341.0,2024-09-21 22:08:04.740232,3,4,4,5,4,"LAGOS,NIGERIA",**Àgbà**\n\n* Ààrẹ\n* Oloye\n* Ọgagun\n* Ọmọ o...,4
301,Ọdun miran tun ti ko fun ilumọọka olorin juju ...,King AbdulRasheed and the Cheif priest are qua...,4,Kini o nipa ija ti wa laarin Oba Abdulrasheed ...,"Yoruba ni ọdọọdun la n ri orogbo, ọdọọdun la n...",3.0,4,A7C7BKFSC8417,3TMSXRD2X61WA2PHT6SXK8GAZ8Y1WT,7.0,...,342.0,2024-09-20 13:44:29.542207,I would rate the given answer a 3. While it me...,1,2,4,2,"LAGOS,NIGERIA","Ìgbà ọ̀tun tún ti wá fún ọ̀gá ọ̀rọ̀ àgbà, olór...",2


In [191]:
import pandas as pd
import numpy as np

# Sample DataFrame
df = df_filtered

# Define the columns to compare for calculating differences
columns_to_compare = ['coherence_gpt', 'consistence_gpt', 'relevance_gpt', 'fluency_gpt',
                      'coherence_gemini', 'consistence_gemini', 'relevance_gemini', 'fluency_gemini']

# Function to compute total difference of each row in a group
def remove_most_different_row(group):
    if len(group) == 4:
        # Create an array to store the total difference for each row
        total_differences = []

        # Iterate over each row to calculate the sum of absolute differences with the other rows
        for i, row in group.iterrows():
            other_rows = group.drop(i)[columns_to_compare]
            row_values = row[columns_to_compare].values
            # Compute the sum of absolute differences between the current row and all other rows

            total_diff = np.abs(other_rows.values - row_values).sum(axis=1).sum()
            total_differences.append(total_diff)
        
        # Find the index of the row with the maximum total difference
        max_diff_idx = group.index[np.argmax(total_differences)]
        
        # Drop the row with the maximum difference
        group = group.drop(max_diff_idx)
    
    return group

# Apply the function to each group of n_sample
df_filtered = df.groupby('n_sample').apply(remove_most_different_row).reset_index(drop=True)

# Display the resulting DataFrame
print(df_filtered)


                                        gpt_prediction  \
0    ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...   
1    ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...   
2    ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...   
3    Ọrọ ti a ti kọ fun yin ni nipa iya kan to pa ọ...   
4    Ọrọ ti a ti kọ fun yin ni nipa iya kan to pa ọ...   
..                                                 ...   
298  Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...   
299  Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...   
300  Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...   
301  Ọdun miran tun ti ko fun ilumọọka olorin juju ...   
302  Ọdun miran tun ti ko fun ilumọọka olorin juju ...   

                                                answer  relevance_gpt  \
0                                                    0              4   
1                                                    0              4   
2                                                    0              5   
3          

/var/folders/6c/7s9j6b710cn7h_bnq4jrn_700000gn/T/ipykernel_2989/4279340566.py:35: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_filtered = df.groupby('n_sample').apply(remove_most_different_row).reset_index(drop=True)


In [204]:
df_filtered['']

,gpt_prediction,answer,relevance_gpt,gpt_question,summary,gpt_grade,consistence_gpt,work_id,assignmentId,task,...,n_sample,date_finish,original_gpt_grade,fluency_gemini,relevance_gemini,fluency_gpt,coherence_gemini,location,gemini_prediction,consistence_gemini
0,ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...,0,4,0,Ijọba apaapọ orilẹede Naijiria ni wọn yoo se i...,0.0,4,A32OWZMIVWENYE,39U1BHVTDLS0SOZ2L515LHO8NHQ3TP,6.0,...,4.0,2024-09-01 15:54:15.977230,0,4,5,4,5,0,**Òrọ Tí A Túmọ̀**\n\n* Igbakeji Aarẹ Naijiria...,4
1,ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...,0,4,0,Ijọba apaapọ orilẹede Naijiria ni wọn yoo se i...,0.0,4,A2ON3JHU4XJ11Y,33F859I566EFQACK04ZMPHJW0F8HBK,10.0,...,4.0,2024-09-01 01:00:53.956401,0,5,4,4,4,0,**Òrọ Tí A Túmọ̀**\n\n* Igbakeji Aarẹ Naijiria...,4
2,ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...,0,5,0,Ijọba apaapọ orilẹede Naijiria ni wọn yoo se i...,0.0,4,A32OWZMIVWENYE,39U1BHVTDLS0SOZ2L515LHO8NHQ3TP,5.0,...,4.0,2024-09-01 15:53:18.326913,0,5,4,4,5,0,**Òrọ Tí A Túmọ̀**\n\n* Igbakeji Aarẹ Naijiria...,4
3,Ọrọ ti a ti kọ fun yin ni nipa iya kan to pa ọ...,0,4,0,Orisirisi iwa isekupani lo n waye ni Naijiria ...,0.0,4,A38NAQ9O99MMIH,3CP1TO84PT29M4STG952ZRD210425M,2.0,...,5.0,2024-09-01 06:57:16.059175,0,4,4,4,4,0,**Awọn Ọrọ Yorùbá:**\n\n* Iroyin (ìròyìn)\n* K...,4
4,Ọrọ ti a ti kọ fun yin ni nipa iya kan to pa ọ...,0,4,0,Orisirisi iwa isekupani lo n waye ni Naijiria ...,0.0,3,AVW7ZRR5PLRFW,3PJUZCGDJ6H396WUZGRKLYV79WF98N,4.0,...,5.0,2024-09-01 13:25:21.954945,0,4,2,2,4,0,**Awọn Ọrọ Yorùbá:**\n\n* Iroyin (ìròyìn)\n* K...,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...,Beeni.,3,Awọn ọrọ ni ki o ṣe iru awọn ohun ti o jẹ ohun...,Wọn bi Aarẹ Muhammadu Buhari ni idile ẹya Fula...,4.0,4,A165LB5CPPQE6M,3W92K5RLWUIIJVQV4JPD1WAD5VGV57,7.0,...,341.0,2024-09-21 22:35:57.353065,I would rate the given answer a 4. It correctl...,3,3,4,2,"Lagos, Nigeria.",**Àgbà**\n\n* Ààrẹ\n* Oloye\n* Ọgagun\n* Ọmọ o...,3
299,Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...,Beeni,3,Maesaiah Thabane ti fi ẹsun ipaniyan kan loril...,Wọn bi Aarẹ Muhammadu Buhari ni idile ẹya Fula...,4.0,3,A165LB5CPPQE6M,3W92K5RLWUIIJVQV4JPD1WAD5VGV57,3.0,...,341.0,2024-09-21 22:30:17.507613,4,4,3,4,4,"Lagos, Nigeria.",**Àgbà**\n\n* Ààrẹ\n* Oloye\n* Ọgagun\n* Ọmọ o...,4
300,Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...,The representative of the pope and Bishop atte...,4,Kini orisun aṣoju Poopu ati aadọta bisọbu ni y...,Wọn bi Aarẹ Muhammadu Buhari ni idile ẹya Fula...,3.0,4,A2A6H80LVCI4U2,3GNA64GUZE5QENUEIKV3V9B5O3Z5QL,1.0,...,341.0,2024-09-21 22:08:04.740232,3,4,4,5,4,"LAGOS,NIGERIA",**Àgbà**\n\n* Ààrẹ\n* Oloye\n* Ọgagun\n* Ọmọ o...,4
301,Ọdun miran tun ti ko fun ilumọọka olorin juju ...,King AbdulRasheed and the Cheif priest are qua...,4,Kini o nipa ija ti wa laarin Oba Abdulrasheed ...,"Yoruba ni ọdọọdun la n ri orogbo, ọdọọdun la n...",3.0,4,A7C7BKFSC8417,3TMSXRD2X61WA2PHT6SXK8GAZ8Y1WT,7.0,...,342.0,2024-09-20 13:44:29.542207,I would rate the given answer a 3. While it me...,1,2,4,2,"LAGOS,NIGERIA","Ìgbà ọ̀tun tún ti wá fún ọ̀gá ọ̀rọ̀ àgbà, olór...",2


In [193]:
import pandas as pd
import numpy as np

# Example DataFrame
df = df_filtered
# Define the columns to compute the difference for
columns_to_compare = ['coherence_gpt', 'consistence_gpt', 'relevance_gpt', 'fluency_gpt',
                      'coherence_gemini', 'consistence_gemini', 'relevance_gemini', 'fluency_gemini']

# Function to remove the row with the highest difference from the others in the group
def remove_most_different_row(group):
    if len(group) == 4:
        # Calculate the pairwise differences for each row in the group
        diffs = []
        for i, row in group.iterrows():
            other_rows = group.drop(i)
            diff = np.abs(other_rows[columns_to_compare].values - row[columns_to_compare].values).sum(axis=1).sum()
            diffs.append(diff)

        # Find the index of the row with the maximum difference
        max_diff_idx = group.index[np.argmax(diffs)]
        
        # Drop the row with the maximum difference
        group = group.drop(max_diff_idx)
    
    return group

# Apply the function to each group of n_sample
df_filtered = df.groupby('n_sample').apply(remove_most_different_row).reset_index(drop=True)

# Display the filtered DataFrame
print(df_filtered)


                                        gpt_prediction  \
0    ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...   
1    ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...   
2    ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...   
3    Ọrọ ti a ti kọ fun yin ni nipa iya kan to pa ọ...   
4    Ọrọ ti a ti kọ fun yin ni nipa iya kan to pa ọ...   
..                                                 ...   
298  Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...   
299  Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...   
300  Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...   
301  Ọdun miran tun ti ko fun ilumọọka olorin juju ...   
302  Ọdun miran tun ti ko fun ilumọọka olorin juju ...   

                                                answer  relevance_gpt  \
0                                                    0              4   
1                                                    0              4   
2                                                    0              5   
3          

/var/folders/6c/7s9j6b710cn7h_bnq4jrn_700000gn/T/ipykernel_2989/709470472.py:29: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  df_filtered = df.groupby('n_sample').apply(remove_most_different_row).reset_index(drop=True)


In [194]:
df_filtered

,gpt_prediction,answer,relevance_gpt,gpt_question,summary,gpt_grade,consistence_gpt,work_id,assignmentId,task,...,n_sample,date_finish,original_gpt_grade,fluency_gemini,relevance_gemini,fluency_gpt,coherence_gemini,location,gemini_prediction,consistence_gemini
0,ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...,0,4,0,Ijọba apaapọ orilẹede Naijiria ni wọn yoo se i...,0.0,4,A32OWZMIVWENYE,39U1BHVTDLS0SOZ2L515LHO8NHQ3TP,6.0,...,4.0,2024-09-01 15:54:15.977230,0,4,5,4,5,0,**Òrọ Tí A Túmọ̀**\n\n* Igbakeji Aarẹ Naijiria...,4
1,ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...,0,4,0,Ijọba apaapọ orilẹede Naijiria ni wọn yoo se i...,0.0,4,A2ON3JHU4XJ11Y,33F859I566EFQACK04ZMPHJW0F8HBK,10.0,...,4.0,2024-09-01 01:00:53.956401,0,5,4,4,4,0,**Òrọ Tí A Túmọ̀**\n\n* Igbakeji Aarẹ Naijiria...,4
2,ti o ti ṣe abẹwo si ibudo awọn to lugbaji omiy...,0,5,0,Ijọba apaapọ orilẹede Naijiria ni wọn yoo se i...,0.0,4,A32OWZMIVWENYE,39U1BHVTDLS0SOZ2L515LHO8NHQ3TP,5.0,...,4.0,2024-09-01 15:53:18.326913,0,5,4,4,5,0,**Òrọ Tí A Túmọ̀**\n\n* Igbakeji Aarẹ Naijiria...,4
3,Ọrọ ti a ti kọ fun yin ni nipa iya kan to pa ọ...,0,4,0,Orisirisi iwa isekupani lo n waye ni Naijiria ...,0.0,4,A38NAQ9O99MMIH,3CP1TO84PT29M4STG952ZRD210425M,2.0,...,5.0,2024-09-01 06:57:16.059175,0,4,4,4,4,0,**Awọn Ọrọ Yorùbá:**\n\n* Iroyin (ìròyìn)\n* K...,4
4,Ọrọ ti a ti kọ fun yin ni nipa iya kan to pa ọ...,0,4,0,Orisirisi iwa isekupani lo n waye ni Naijiria ...,0.0,3,AVW7ZRR5PLRFW,3PJUZCGDJ6H396WUZGRKLYV79WF98N,4.0,...,5.0,2024-09-01 13:25:21.954945,0,4,2,2,4,0,**Awọn Ọrọ Yorùbá:**\n\n* Iroyin (ìròyìn)\n* K...,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...,Beeni.,3,Awọn ọrọ ni ki o ṣe iru awọn ohun ti o jẹ ohun...,Wọn bi Aarẹ Muhammadu Buhari ni idile ẹya Fula...,4.0,4,A165LB5CPPQE6M,3W92K5RLWUIIJVQV4JPD1WAD5VGV57,7.0,...,341.0,2024-09-21 22:35:57.353065,I would rate the given answer a 4. It correctl...,3,3,4,2,"Lagos, Nigeria.",**Àgbà**\n\n* Ààrẹ\n* Oloye\n* Ọgagun\n* Ọmọ o...,3
299,Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...,Beeni,3,Maesaiah Thabane ti fi ẹsun ipaniyan kan loril...,Wọn bi Aarẹ Muhammadu Buhari ni idile ẹya Fula...,4.0,3,A165LB5CPPQE6M,3W92K5RLWUIIJVQV4JPD1WAD5VGV57,3.0,...,341.0,2024-09-21 22:30:17.507613,4,4,3,4,4,"Lagos, Nigeria.",**Àgbà**\n\n* Ààrẹ\n* Oloye\n* Ọgagun\n* Ọmọ o...,4
300,Ààrẹ Muhammadu Buhari jẹ ọmọ ẹgbẹ Fulani kan t...,The representative of the pope and Bishop atte...,4,Kini orisun aṣoju Poopu ati aadọta bisọbu ni y...,Wọn bi Aarẹ Muhammadu Buhari ni idile ẹya Fula...,3.0,4,A2A6H80LVCI4U2,3GNA64GUZE5QENUEIKV3V9B5O3Z5QL,1.0,...,341.0,2024-09-21 22:08:04.740232,3,4,4,5,4,"LAGOS,NIGERIA",**Àgbà**\n\n* Ààrẹ\n* Oloye\n* Ọgagun\n* Ọmọ o...,4
301,Ọdun miran tun ti ko fun ilumọọka olorin juju ...,King AbdulRasheed and the Cheif priest are qua...,4,Kini o nipa ija ti wa laarin Oba Abdulrasheed ...,"Yoruba ni ọdọọdun la n ri orogbo, ọdọọdun la n...",3.0,4,A7C7BKFSC8417,3TMSXRD2X61WA2PHT6SXK8GAZ8Y1WT,7.0,...,342.0,2024-09-20 13:44:29.542207,I would rate the given answer a 3. While it me...,1,2,4,2,"LAGOS,NIGERIA","Ìgbà ọ̀tun tún ti wá fún ọ̀gá ọ̀rọ̀ àgbà, olór...",2


In [195]:
# df_filtered.to_excel("/Users/itaimondshine/PycharmProjects/NLP/eval_metrics/evaluations/results/human/documents/ukrinian.xlsx")

In [196]:
import pandas as pd

# Assuming df is your first DataFrame

# Define the columns for GPT and Gemini
gpt_columns = ['coherence_gpt', 'consistence_gpt', 'relevance_gpt', 'fluency_gpt']
gemini_columns = ['coherence_gemini', 'consistence_gemini', 'relevance_gemini', 'fluency_gemini']

# Create an empty list to store the results
rows = []

# Group by 'n_sample' and 'model'
for n_sample, group in df_filtered.groupby('n_sample'):
    # For GPT
    gpt_data = {
        'coherence': group['coherence_gpt'].tolist(),
        'consistency': group['consistence_gpt'].tolist(),
        'relevance': group['relevance_gpt'].tolist(),
        'fluency': group['fluency_gpt'].tolist(),
        'n_sample': n_sample,
        'model': 'gpt'
    }

    # For Gemini
    gemini_data = {
        'coherence': group['coherence_gemini'].tolist(),
        'consistency': group['consistence_gemini'].tolist(),
        'relevance': group['relevance_gemini'].tolist(),
        'fluency': group['fluency_gemini'].tolist(),
        'n_sample': n_sample,
        'model': 'gemini'
    }

    # Append the rows for both GPT and Gemini
    rows.append(gpt_data)
    rows.append(gemini_data)

# Convert the list of dictionaries to a DataFrame
list_df = pd.DataFrame(rows)

# Display the final DataFrame
del list_df['model']
del list_df['n_sample']
print(list_df)

     coherence consistency  relevance    fluency
0    [5, 4, 5]   [4, 4, 4]  [4, 4, 5]  [4, 4, 4]
1    [5, 4, 5]   [4, 4, 4]  [5, 4, 4]  [4, 5, 5]
2    [4, 4, 4]   [4, 3, 5]  [4, 4, 5]  [4, 2, 4]
3    [4, 4, 4]   [4, 3, 4]  [4, 2, 5]  [4, 4, 4]
4    [4, 4, 3]   [4, 4, 3]  [4, 3, 3]  [2, 2, 3]
..         ...         ...        ...        ...
233     [5, 4]      [5, 4]     [5, 4]     [5, 4]
234  [4, 4, 4]   [4, 3, 4]  [3, 3, 4]  [4, 4, 5]
235  [2, 4, 4]   [3, 4, 4]  [3, 3, 4]  [3, 4, 4]
236     [4, 4]      [4, 3]     [4, 4]     [4, 4]
237     [2, 2]      [2, 3]     [2, 3]     [1, 2]

[238 rows x 4 columns]


In [197]:
df_single_column = list_df.melt(value_name='stacked').sort_index()
# 

In [198]:
df_single_column

,variable,stacked
0,coherence,"[5, 4, 5]"
1,coherence,"[5, 4, 5]"
2,coherence,"[4, 4, 4]"
3,coherence,"[4, 4, 4]"
4,coherence,"[4, 4, 3]"
...,...,...
947,fluency,"[5, 4]"
948,fluency,"[4, 4, 5]"
949,fluency,"[3, 4, 4]"
950,fluency,"[4, 4]"


In [199]:
# df_single_column = df_single_column.melt(value_name='stacked').sort_index()

# Drop the 'variable' column that was holding the original column names
df_single_column = df_single_column[['stacked']].reset_index(drop=True)

df_single_column = df_single_column.dropna()

In [200]:
df_single_column['stacked'] = df_single_column['stacked'].apply(lambda r: print(r))

[5, 4, 5]
[5, 4, 5]
[4, 4, 4]
[4, 4, 4]
[4, 4, 3]
[4, 3, 3]
[4, 5, 4]
[4, 4, 3]
[4, 5, 4]
[5, 4, 2]
[4, 4, 4]
[4, 4, 4]
[2, 4, 4]
[4, 4, 3]
[4, 4, 3]
[4, 3, 4]
[3, 4, 4]
[3, 4, 5]
[4, 3, 4]
[3, 4, 4]
[5, 4]
[4, 4]
[3, 5, 2]
[3, 4, 4]
[4, 3]
[4, 4]
[5, 3]
[5, 4]
[5, 4, 4]
[4, 2, 5]
[4, 5, 3]
[4, 5, 4]
[4, 4]
[3, 4]
[3, 3]
[4, 5]
[4, 4, 3]
[4, 4, 3]
[4, 3]
[4, 3]
[4, 4, 3]
[4, 3, 4]
[4, 4, 4]
[4, 5, 5]
[4, 4, 4]
[4, 4, 4]
[4, 4]
[5, 4]
[4, 4, 4]
[4, 4, 4]
[1, 2, 3]
[4, 4, 5]
[4, 5, 4]
[5, 3, 4]
[4, 4]
[4, 4]
[3, 3, 4]
[3, 3, 4]
[3, 4]
[3, 4]
[5, 4]
[4, 5]
[4, 4, 3]
[4, 4, 3]
[4, 4, 4]
[4, 5, 5]
[3, 4, 3]
[5, 4, 3]
[2, 3, 2]
[4, 4, 3]
[4, 5, 5]
[4, 4, 4]
[4, 4, 5]
[4, 4, 5]
[4, 3, 4]
[4, 3, 4]
[3, 3]
[3, 4]
[4, 5]
[4, 5]
[3, 4]
[4, 2]
[4, 4]
[4, 4]
[4, 2, 4]
[4, 4, 4]
[4, 3]
[4, 3]
[3, 3, 3]
[4, 3, 4]
[4, 4, 4]
[5, 5, 5]
[4, 5, 5]
[5, 5, 5]
[5, 5]
[5, 5]
[5, 5, 5]
[5, 5, 5]
[4, 4]
[5, 5]
[4, 4, 4]
[5, 5, 5]
[5, 5]
[5, 5]
[5, 5]
[5, 5]
[5, 5, 5]
[5, 5, 5]
[4, 3, 4]
[4, 3, 3]
[5, 5]
[1, 1]


In [201]:
import pandas as pd
import numpy as np

# Assuming list_df is the DataFrame with coherence, consistency, relevance, and fluency

# Melt the DataFrame to stack all values in a single column
df_single_column = list_df.melt(value_name='stacked').sort_index()

# Drop the 'variable' column that was holding the original column names
df_single_column = df_single_column[['stacked']].reset_index(drop=True)

# Function to replace NaN with None, sort the list, and pad to 3 elements
def clean_and_sort(lst):
    # Replace NaN values with None
    lst = [None if pd.isna(x) else x for x in lst]
    
    # Separate None values
    non_none_values = [x for x in lst if x is not None]
    none_values = [x for x in lst if x is None]
    
    # Sort the non-None values
    non_none_values = sorted(non_none_values)
    
    # Combine sorted non-None values and None values, and truncate to 3
    cleaned_lst = non_none_values + none_values
    cleaned_lst = cleaned_lst[:3]
    
    # If the list has fewer than 3 elements, pad with None
    return cleaned_lst + [None] * (3 - len(cleaned_lst))

# Apply the function to ensure each list has 3 elements, sorted, and NaN values replaced
df_single_column['stacked'] = df_single_column['stacked'].apply(clean_and_sort)

# Display the final DataFrame
print(df_single_column)


          stacked
0       [4, 5, 5]
1       [4, 5, 5]
2       [4, 4, 4]
3       [4, 4, 4]
4       [3, 4, 4]
..            ...
947  [4, 5, None]
948     [4, 4, 5]
949     [3, 4, 4]
950  [4, 4, None]
951  [1, 2, None]

[952 rows x 1 columns]


In [202]:
df_single_column['stacked']

0         [4, 5, 5]
1         [4, 5, 5]
2         [4, 4, 4]
3         [4, 4, 4]
4         [3, 4, 4]
           ...     
947    [4, 5, None]
948       [4, 4, 5]
949       [3, 4, 4]
950    [4, 4, None]
951    [1, 2, None]
Name: stacked, Length: 952, dtype: object

In [107]:
import pandas as pd
import numpy as np

# Sample DataFrame for demonstration
# Replace this with your actual DataFrame
# df = pd.DataFrame({
#     'stacked': [
#         [1, 1, 2],  # This row should be filtered out (has two 1s)
#         [3, 4, 5],  # This row should be included (no duplicates)
#         [2, 2, 3],  # This row should be filtered out (has two 2s)
#         [4, 5, 6],  # This row should be included (no duplicates)
#         [7, None, 7]  # This row should be filtered out (has two 7s)
#     ]
# })

# Function to check if a list has at least two identical numbers
def has_identical_numbers(lst):
    if lst is None:  # Handle None values if they exist
        return False
    # Remove None values
    lst = [x for x in lst if x is not None]
    # Check for duplicates
    number_counts = {n: lst.count(n) for n in lst}
    # Return True if any number has a count of 2 or more
    return any(count > 1 for count in number_counts.values())

# Apply the function to filter the DataFrame
df_filtered = df_single_column[df_single_column['stacked'].apply(has_identical_numbers)]

# Display the filtered DataFrame
print(df_filtered)


       stacked
0    [4, 4, 4]
1    [2, 3, 3]
2    [4, 4, 5]
3    [4, 4, 4]
4    [3, 3, 4]
..         ...
947  [3, 3, 3]
948  [3, 3, 3]
949  [3, 3, 3]
950  [3, 3, 3]
951  [3, 3, 3]

[842 rows x 1 columns]


In [126]:
len(df_single_column)

944

In [212]:
df_filtered.to_excel("/Users/itaimondshine/PycharmProjects/NLP/eval_metrics/evaluations/results/human/documents/final/yoruba.xlsx")

In [203]:
# Calculate agreement rate

import numpy as np
df =df_single_column

# Convert the column of lists into a 3xN matrix
matrix = np.array(df['stacked'].tolist()).T

# Function to replace None with *
def replace_none_with_star(x):
    return " ".join(str(i) if i is not None else "*" for i in x)

# Convert each row of the matrix to the desired format
formatted_data = tuple(replace_none_with_star(row) for row in matrix)


try:
    import numpy as np
except ImportError:
    np = None


def nominal_metric(a, b):
    return a != b


def interval_metric(a, b):
    return (a-b)**2


def ratio_metric(a, b):
    return ((a-b)/(a+b))**2


def krippendorff_alpha(data, metric=interval_metric, force_vecmath=False, convert_items=float, missing_items=None):
    '''
    Calculate Krippendorff's alpha (inter-rater reliability):
    
    data is in the format
    [
        {unit1:value, unit2:value, ...},  # coder 1
        {unit1:value, unit3:value, ...},   # coder 2
        ...                            # more coders
    ]
    or 
    it is a sequence of (masked) sequences (list, numpy.array, numpy.ma.array, e.g.) with rows corresponding to coders and columns to items
    
    metric: function calculating the pairwise distance
    force_vecmath: force vector math for custom metrics (numpy required)
    convert_items: function for the type conversion of items (default: float)
    missing_items: indicator for missing items (default: None)
    '''
    
    # number of coders
    m = len(data)
    
    # set of constants identifying missing values
    if missing_items is None:
        maskitems = []
    else:
        maskitems = list(missing_items)
    if np is not None:
        maskitems.append(np.ma.masked_singleton)
    
    # convert input data to a dict of items
    units = {}
    for d in data:
        try:
            # try if d behaves as a dict
            diter = d.items()
        except AttributeError:
            # sequence assumed for d
            diter = enumerate(d)
            
        for it, g in diter:
            if g not in maskitems:
                try:
                    its = units[it]
                except KeyError:
                    its = []
                    units[it] = its
                its.append(convert_items(g))


    units = dict((it, d) for it, d in units.items() if len(d) > 1)  # units with pairable values
    n = sum(len(pv) for pv in units.values())  # number of pairable values
    
    if n == 0:
        raise ValueError("No items to compare.")
    
    np_metric = (np is not None) and ((metric in (interval_metric, nominal_metric, ratio_metric)) or force_vecmath)
    
    Do = 0.
    for grades in units.values():
        if np_metric:
            gr = np.asarray(grades)
            Du = sum(np.sum(metric(gr, gri)) for gri in gr)
        else:
            Du = sum(metric(gi, gj) for gi in grades for gj in grades)
        Do += Du/float(len(grades)-1)
    Do /= float(n)

    if Do == 0:
        return 1.

    De = 0.
    for g1 in units.values():
        if np_metric:
            d1 = np.asarray(g1)
            for g2 in units.values():
                De += sum(np.sum(metric(d1, gj)) for gj in g2)
        else:
            for g2 in units.values():
                De += sum(metric(gi, gj) for gi in g1 for gj in g2)
    De /= float(n*(n-1))

    return 1.-Do/De if (Do and De) else 1.



missing = '*' # indicator for missing values
array = [d.split() for d in formatted_data]  # convert to 2D list of string items

print("nominal metric: %.3f" % krippendorff_alpha(array, nominal_metric, missing_items=missing))
print("interval metric: %.3f" % krippendorff_alpha(array, interval_metric, missing_items=missing))

nominal metric: 0.206
interval metric: 0.496


In [131]:
# df_filtered = pd.read_excel("/Users/itaimondshine/PycharmProjects/NLP/eval_metrics/evaluations/results/human/documents/final/turkish.xlsx")

In [132]:
import pandas as pd

# Assuming df is your initial DataFrame
# Extract relevant columns for GPT and Gemini
gpt_columns = ['coherence_gpt', 'consistence_gpt', 'relevance_gpt', 'fluency_gpt']
gemini_columns = ['coherence_gemini', 'consistence_gemini', 'relevance_gemini', 'fluency_gemini']

# Create an empty list to store the results
rows = []

# Group by 'n_sample'
for n_sample, group in df_filtered.groupby('n_sample'):
    # Calculate the averages for GPT
    gpt_avg = group[gpt_columns].mean()
    # Calculate the averages for Gemini
    gemini_avg = group[gemini_columns].mean()
    
    # Add results to the rows list
    rows.append({
        'coherence': gpt_avg['coherence_gpt'],
        'consistency': gpt_avg['consistence_gpt'],
        'relevance': gpt_avg['relevance_gpt'],
        'fluency': gpt_avg['fluency_gpt'],
        'n_sample': n_sample,
        'model': 'gpt'
    })
    
    rows.append({
        'coherence': gemini_avg['coherence_gemini'],
        'consistency': gemini_avg['consistence_gemini'],
        'relevance': gemini_avg['relevance_gemini'],
        'fluency': gemini_avg['fluency_gemini'],
        'n_sample': n_sample,
        'model': 'gemini'
    })

# Convert the list of dictionaries to a DataFrame
result_df = pd.DataFrame(rows)

# Display the final DataFrame
print(result_df)


     coherence  consistency  relevance   fluency  n_sample   model
0     3.500000     2.750000       3.75  3.750000         0     gpt
1     3.500000     3.000000       3.25  4.250000         0  gemini
2     3.000000     3.000000       3.00  3.500000         1     gpt
3     3.000000     3.000000       3.50  4.000000         1  gemini
4     3.000000     3.250000       3.00  4.000000         2     gpt
..         ...          ...        ...       ...       ...     ...
323   3.000000     3.000000       3.00  3.000000       299  gemini
324   3.000000     3.000000       3.00  3.000000       301     gpt
325   3.000000     3.000000       3.00  3.000000       301  gemini
326   3.666667     2.666667       3.00  4.666667       302     gpt
327   4.666667     4.333333       4.00  4.000000       302  gemini

[328 rows x 6 columns]


In [137]:
result_df.to_csv("/Users/itaimondshine/PycharmProjects/NLP/eval_metrics/evaluations/results/human/documents/final/averages/turkish.csv")

In [138]:
result_df

,coherence,consistency,relevance,fluency,n_sample,model
0,3.500000,2.750000,3.75,3.750000,0,gpt
1,3.500000,3.000000,3.25,4.250000,0,gemini
2,3.000000,3.000000,3.00,3.500000,1,gpt
3,3.000000,3.000000,3.50,4.000000,1,gemini
4,3.000000,3.250000,3.00,4.000000,2,gpt
...,...,...,...,...,...,...
323,3.000000,3.000000,3.00,3.000000,299,gemini
324,3.000000,3.000000,3.00,3.000000,301,gpt
325,3.000000,3.000000,3.00,3.000000,301,gemini
326,3.666667,2.666667,3.00,4.666667,302,gpt


In [ ]:
# Combine with LLm as a judeg

In [152]:
llm = pd.read_csv("/Users/itaimondshine/PycharmProjects/NLP/eval_metrics/evaluations/results/llm/hebrew.csv")

In [153]:
llm.columns

Index(['Unnamed: 0', 'n_sample', 'coherence_gemini_llm',
       'consistency_gemini_llm', 'fluency_gemini_llm', 'relevance_gemini_llm',
       'coherence_gpt_llm', 'consistency_gpt_llm', 'fluency_gpt_llm',
       'relevance_gpt_llm'],
      dtype='object')

In [154]:
import pandas as pd

# Assuming df1 is the first DataFrame and df2 is the new one with grades for each n_sample
# Reshape df2 to have one row per n_sample and model (GPT/Gemini)
df2_gpt = llm[['n_sample', 'coherence_gpt_llm', 'consistency_gpt_llm', 'fluency_gpt_llm', 'relevance_gpt_llm']].copy()
df2_gpt['model'] = 'gpt'
df2_gpt.rename(columns={
    'coherence_gpt_llm': 'coherence_llm',
    'consistency_gpt_llm': 'consistency_llm',
    'fluency_gpt_llm': 'fluency_llm',
    'relevance_gpt_llm': 'relevance_llm'
}, inplace=True)

df2_gemini = llm[['n_sample', 'coherence_gemini_llm', 'consistency_gemini_llm', 'fluency_gemini_llm', 'relevance_gemini_llm']].copy()
df2_gemini['model'] = 'gemini'
df2_gemini.rename(columns={
    'coherence_gemini_llm': 'coherence_llm',
    'consistency_gemini_llm': 'consistency_llm',
    'fluency_gemini_llm': 'fluency_llm',
    'relevance_gemini_llm': 'relevance_llm'
}, inplace=True)

# Concatenate the GPT and Gemini DataFrames
df2_long = pd.concat([df2_gpt, df2_gemini], ignore_index=True)

# Merge df1 and df2_long based on n_sample and model
merged_df = pd.merge(result_df, df2_long, on=['n_sample', 'model'], how='left')
# 
# Display the merged DataFrame
# print(merged_df)


In [155]:
merged_df['coherence_llm'] = merged_df['coherence_llm'].apply(lambda r: r+1)

In [156]:
import pandas as pd
from scipy.stats import pearsonr

# Assuming merged_df is your DataFrame with all the relevant columns

# Initialize an empty list to store results
results = []

# List of column pairs to compare
columns = ['coherence', 'consistency', 'relevance', 'fluency']

# Loop through each pair and calculate Pearson correlation and p-value
for col in columns:
    r, p_value = pearsonr(merged_df[col], merged_df[f'{col}_llm'])
    results.append({
        'criterion': col,
        'pearson_correlation': r,
        'p_value': p_value
    })

# Convert results to a DataFrame for better visualization
correlation_df = pd.DataFrame(results)

# Display the results
print(correlation_df)


     criterion  pearson_correlation   p_value
0    coherence            -0.069384  0.492749
1  consistency             0.122951  0.222964
2    relevance             0.162192  0.106914
3      fluency             0.011746  0.907659


In [33]:
# Correlation with metrics 

In [34]:
merged_df['coherence']

0      4.000000
1      4.333333
2      3.333333
3      3.333333
4      3.333333
         ...   
181    3.000000
182    4.333333
183    4.333333
184    4.000000
185    3.666667
Name: coherence, Length: 186, dtype: float64

In [35]:
merged_df['coherence_llm']

0      2.666667
1      4.000000
2      1.666667
3      0.666667
4      2.333333
         ...   
181    3.333333
182    3.000000
183    4.000000
184    3.000000
185    4.000000
Name: coherence_llm, Length: 186, dtype: float64

In [57]:
# Agreement Rate